# L4: Dependency parsing

In this lab, you will implement a simplified version of the dependency parser presented by [Glavaš and Vulić (2021)](http://dx.doi.org/10.18653/v1/2021.eacl-main.270). This parser consists of a transformer encoder followed by a bi-affine layer that computes arc scores for all pairs of words. These scores are then used as logits in a classifier that predicts the position of the head of each word. In contrast to the parser described in the paper, yours will only support unlabelled parsing, that is, you will not implement what the authors call a *relation classifier*. As the encoder, you will use the [uncased DistilBERT base model](https://huggingface.co/docs/transformers/model_doc/distilbert) from the [Transformers](https://huggingface.co/docs/transformers/main/en/index) library, even though every other BERT-based encoder should work equally well.

In [1]:
!pip3 install transformers conllu
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Dataset

The data for this lab comes from the English Web Treebank from the [Universal Dependencies Project](http://universaldependencies.org). To read the data, we use the [CoNLL-U Parser](https://pypi.org/project/conllu/) library. The code in the next cell defines a PyTorch [Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) wrapper for the data.

In [6]:
import conllu

from torch.utils.data import Dataset

class ParserDataset(Dataset):

    def __init__(self, filename):
        super().__init__()
        self.items = []
        with open(filename, 'rt', encoding='utf-8') as fp:
            for tokens in conllu.parse_incr(fp):
                self.items.append([(t['form'], t['head']) for t in tokens if isinstance(t['id'], int)])

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        return self.items[idx]

We can now load the training data:

In [7]:
TRAIN_DATA = ParserDataset('en_ewt-ud-train.conllu')

Our data consists of **parsed sentences**. A parsed sentence is represented as a list of pairs. The first component of each pair (a string) represents a word; the second component (an integer) specifies the position of the word’s head, i.e., its parent in the dependency tree. Note that word positions are numbered starting at&nbsp;1; the head position&nbsp;0 represents the root of the tree.

Run the next cell to see an example sentence:

In [8]:
EXAMPLE_WORDS: tuple[str] = list(zip(*TRAIN_DATA[531]))[0]
EXAMPLE_HEADS: tuple[int] = list(zip(*TRAIN_DATA[531]))[1]

EXAMPLE_WORDS, EXAMPLE_HEADS

(('I', 'like', 'yuor', 'blog', '.'), (2, 0, 4, 2, 2))

The example sentence consists of five whitespace-separated words, including the final punctuation mark. The head of the pronoun *I* is the word at position&nbsp;2 – the verb *like*. The head of the word *like* is the root of the tree (position&nbsp;0). The dependents of *like* are *I* (position&nbsp;1), the noun *blog* (position&nbsp;4), and the final punctuation mark. Note that the pronoun *your* (position&nbsp;3) is misspelled as *yuor*.

## Problem 1: Tokenisation (2 points)

To feed parsed sentences to DistilBERT, we need to tokenise them and encode the resulting tokens as integers in the model vocabulary. We start by loading the DistilBERT tokeniser using the [Auto classes](https://huggingface.co/docs/transformers/v4.37.2/en/model_doc/auto):

In [9]:
from transformers import AutoTokenizer
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast

TOKENIZER: PreTrainedTokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

We can call the tokeniser on the example sentence as shown in the next cell. Note that we use the *is_split_into_words* keyword argument to indicate that the input is already pre-tokenised (split on whitespace).

In [10]:
EXAMPLE_TOKENS = TOKENIZER(EXAMPLE_WORDS, is_split_into_words=True)

The output of the tokeniser is an object of class [`BatchEncoding`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding). The code in the following cell shows the list of tokens:

In [11]:
[TOKENIZER.decode(i) for i in EXAMPLE_TOKENS.input_ids]

['[CLS]', 'i', 'like', 'yu', '##or', 'blog', '.', '[SEP]']

As you can see, the tokeniser adds the special tokens `[CLS]` and `[SEP]` and splits unknown words (here: the misspelled word *yuor*) into sub-word tokens. We will need to keep track of which tokens correspond to which word. To achieve that, we can use the method [`word_to_tokens()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_to_tokens), which gets us the encoded token span (an object of class [`TokenSpan`](https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.TokenSpan)) corresponding to a word in a sequence of the input batch:

In [12]:
from transformers import TokenSpan

for i, word in enumerate(EXAMPLE_WORDS):
    token_span: TokenSpan = EXAMPLE_TOKENS.word_to_tokens(i)
    print(token_span, '->', word)

TokenSpan(start=1, end=2) -> I
TokenSpan(start=2, end=3) -> like
TokenSpan(start=3, end=5) -> yuor
TokenSpan(start=5, end=6) -> blog
TokenSpan(start=6, end=7) -> .


Your first task is to code a function `encode()` that takes a tokeniser and a list of sentences and returns the tokeniser&rsquo;s encoded input as well as the corresponding token spans. The following cell contains skeleton code for this function.

In [13]:
from transformers import BatchEncoding
from typing import Iterable
import torch

def encode(tokenizer: PreTrainedTokenizer | PreTrainedTokenizerFast, sentences: list[list[tuple[str, int]]]) -> tuple[BatchEncoding, list[list[TokenSpan]]]:
    '''
    Returns tokeniser's encoded input and corresponding token spans
    '''

    split_sentence_list = [[word for word, _ in sentence] for sentence in sentences]

    tokens: BatchEncoding = tokenizer(split_sentence_list, truncation=True, padding=True, is_split_into_words=True, return_tensors='pt')
    tokens.input_ids = torch.tensor(tokens.input_ids)

    token_spans_list: list[list[TokenSpan]] = []

    for i in range(len(split_sentence_list)):
        token_spans = []
        for j in range(len(split_sentence_list[i])):
            token_spans.append(tokens.word_to_tokens(i, j))
        token_spans_list.append(token_spans)

    return tokens, token_spans_list

Implement this function to match the following specification:

**encode** (*tokenizer*, *sentences*):

> Uses the specified *tokenizer* to encode a batch of parsed sentences (*sentences*). Returns a pair consisting of a [`BatchEncoding`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding) and a matching batch of token spans (as explained above). The [`BatchEncoding`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding) is the standard batch encoding, but with tensors instead of lists of Python integers. Sentences have been truncated to the maximum acceptable input length.

### 🤞 Test your code

To test you code, call `encode()` on a small number of sentences and check that output matches your expectations.

In [14]:
# Test the encode() function
sentences = TRAIN_DATA[531:534]
encoded_batch = encode(TOKENIZER, sentences)

print(encoded_batch[0]['input_ids'])

for token_spans in encoded_batch[1]:
    print(token_spans)


tensor([[ 101, 1045, 2066, 9805, 2953, 9927, 1012,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 3531, 4638, 2041, 2026, 2040, 2292, 1996, 3899, 2041, 9927, 1012,
          102,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 4931, 2008, 1005, 1055, 1037, 2307, 9927, 1010, 1045, 2036, 2031,
         1037, 2609, 2055, 3050, 3349, 3784, 5306,  102]])
[TokenSpan(start=1, end=2), TokenSpan(start=2, end=3), TokenSpan(start=3, end=5), TokenSpan(start=5, end=6), TokenSpan(start=6, end=7)]
[TokenSpan(start=1, end=2), TokenSpan(start=2, end=3), TokenSpan(start=3, end=4), TokenSpan(start=4, end=5), TokenSpan(start=5, end=6), TokenSpan(start=6, end=7), TokenSpan(start=7, end=8), TokenSpan(start=8, end=9), TokenSpan(start=9, end=10), TokenSpan(start=10, end=11), TokenSpan(start=11, end=12)]
[TokenSpan(start=1, end=2), TokenSpan(start=2, end=3), TokenSpan(start=3, end=5), TokenSpan(start=5, end=6), TokenSpan(start=6, end=7), TokenSp

<ipython-input-13-76e280732ca7>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens.input_ids = torch.tensor(tokens.input_ids)


## Problem 2: Merging tokens (4 points)

DistilBERT gives us a representation for each *token* in a sentence. To compute scores between pairs of *words*, we will need to combine the token representations that correspond to each word. A standard strategy for this is to take their element-wise mean. The next cell contains skeleton code for a function `merge_tokens()` that implements this strategy.

In [15]:
from torch import Tensor

def merge_tokens(tokens: Tensor, token_spans_list: list[list[TokenSpan]]):
    '''
    tokens: Tensor[batch_size, num_tokens, hidden_dim]
    token_spans: list[TokenSpan]
    return word_level_representations: Tensor[batch_size, max_num_words, hidden_dim]
    '''
    batch_size, num_tokens, hidden_dim = tokens.shape
    max_num_words = max(len(token_spans) for token_spans in token_spans_list)
    word_level_representations = tokens.new_zeros(batch_size, max_num_words, hidden_dim)

    for i, token_spans in enumerate(token_spans_list):  # Iterate through each sentence in the batch
        for j, (start, end) in enumerate(token_spans):  # Iterate through each word's span in the sentence
            # Calculate the mean of the tokens for this word. Note that `end` is exclusive, so we add 1 to include it.
            word_tensor = tokens[i, start:end].mean(dim=0)
            # Assign the mean tensor to the correct position in the word-level representations tensor.
            word_level_representations[i, j] = word_tensor


    return word_level_representations

Implement this function to match the following specification:

**merge_tokens** (*tokens*, *token_spans*)

> Takes a batch of token vectors (*tokens*) and a list of matching token spans (*token_spans*) and returns a batch of word-level representations, computed using the element-wise mean. The token vectors are a tensor of shape (*batch_size*, *num_tokens*, *hidden_dim*), where *hidden_dim* is the dimensionality of the DistilBERT representations. The token spans are a nested list containing integer pairs, as computed in Problem&nbsp;1. The result is a tensor of shape (*batch_size*, *max_num_words*, *hidden_dim*), where *max_num_words* denotes the maximum number of words in any sentence in the batch. Entries corresponding to padding are represented by the zero vector of size *hidden_dim*.

### 🤞 Test your code

To test you code, create a sample input to `merge_tokens()` and check that the output matches your expectations.

In [16]:
from torch import Tensor, rand

tokens = rand(1, 1000, 3)

# Test the encode() function
sentences = TRAIN_DATA[531:532]
_, token_spans_list = encode(TOKENIZER, sentences)

out = merge_tokens(tokens, token_spans_list)

print(out)

tensor([[[0.9788, 0.2283, 0.6982],
         [0.8639, 0.7545, 0.9967],
         [0.3531, 0.5917, 0.7088],
         [0.8670, 0.0533, 0.1570],
         [0.8822, 0.3684, 0.6935]]])


<ipython-input-13-76e280732ca7>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens.input_ids = torch.tensor(tokens.input_ids)


## Problem 3: Biaffine layer (2 points)

Your next task is to implement the bi-affine layer. Given matrices $X \in \mathbb{R}^{m \times d}$ and $X' \in \mathbb{R}^{n \times d}$, this layer computes a matrix $Y \in \mathbb{R}^{m \times n}$ as

$$
Y = X W X'{}^\top + b
$$

where $W \in \mathbb{R}^{d \times d}$ and $b \in \mathbb{R}$ are learnable weight and bias parameters. In the context of the dependency parser, the matrices $X$ and $X'$ hold the word representations of all dependents and all heads in the input sentence, and the entries of the matrix $Y$ are interpreted as scores of possible dependency arcs. More specifically, the entry $Y_{ij}$ represents the score of an arc from a head word at position&nbsp;$j$ to a dependent at position&nbsp;$i$.


The following cell contains skeleton code for the implementation of the bi-affine layer. Implement this layer according to the specification above.

In [17]:
import torch.nn as nn
import torch

class Biaffine(nn.Module):

    def __init__(self, encoder_dim):
        super().__init__()
        # Weight matrix for the bi-affine transformation, shape: (d, d)
        self.weight = nn.Parameter(torch.rand(encoder_dim, encoder_dim))
        # Bias term
        self.bias = nn.Parameter(torch.rand(1))

    def forward(self, x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
        # x1 shape: (batch_size, m, d), x2 shape: (batch_size, n, d)
        # We perform a batch matrix multiplication between x1 and the weight, resulting in a shape of (batch_size, m, d)
        y = x1 @ self.weight @ x2.transpose(1, 2) + self.bias

        """ # Then we perform another batch matrix multiplication, this time with the transpose of x2.
        # For this, we need to transpose x2 to shape (batch_size, d, n) to align the dimensions for matrix multiplication.
        # The resulting shape of the operation will be (batch_size, m, n), aligning with our target output shape.
        y = torch.bmm(x1_transform, x2.transpose(1, 2))

        # Adding the bias term to every element in the output matrix.
        # Since the bias is a scalar, it's automatically broadcasted across all dimensions.
        y += self.bias """

        return y

**⚠️ Note that your implementation should be able to handle *batches* of input sentences.**

### 🤞 Test your code

To test you code, create a sample input to the bi-affine layer as well as suitable weights and biases and check that the output of the `forward` method matches your expectations

In [18]:
word_level_representations = rand(1, 12, 3)
word_level_representations2 = rand(1, 13, 3)
biaffine = Biaffine(3)
out = biaffine(word_level_representations, word_level_representations2)
print(out.shape) # should be (1, 12, 13)

torch.Size([1, 12, 13])


## Problem 4: Parser (6 points)

We are now ready to put the two main components of the parser together: the encoder (DistilBert) and the bi-affine layer that computes the arc scores. We also add a dropout layer between the two components.

The following code cell contains skeleton code for the parsing model with the `init()` method already complete. Your task is to implement the `forward()` method. If you are unsure how things should be wired up, have another look at the slides.

In [19]:
import torch.nn as nn
import torch

from transformers import DistilBertModel, DistilBertPreTrainedModel

class DistilBertForParsing(DistilBertPreTrainedModel):

    def __init__(self, config, dropout=0.1):
        super().__init__(config)
        self.config = config
        self.distilbert = DistilBertModel(config)
        self.dropout = nn.Dropout(dropout)
        self.biaffine = Biaffine(config.hidden_size)

    def forward(self, encoded_input: BatchEncoding, token_spans: list[list[TokenSpan]]) -> torch.Tensor:
        '''
        returns: torch.Tensor[batch_size, num_words, num_words + 1]
        '''
        # Encode the input
        bert_outputs = self.distilbert(**encoded_input)
        # Merge the tokens into word-level representations
        word_representations = self.dropout(merge_tokens(bert_outputs.last_hidden_state, token_spans))
        # Apply the bi-affine transformation
        scores = self.biaffine(word_representations, word_representations)
        batch_size, num_words, _ = scores.shape
        root_scores = torch.cat([torch.zeros(batch_size, num_words, 1, device=scores.device), scores], dim=2)

        return root_scores


Implement the `forward()` method to match the following specification:

**forward** (*encoded_input*, *token_spans*)

> Takes a tokeniser-encoded batch of sentences (*encoded_input*, of type `BatchEncoding`) and a corresponding nested list of token spans (*token_spans*) and returns a tensor with scores for each pair of words. More specifically, the output tensor $Y$ has shape (*batch_size*, *num_words*, *num_words+1*), where the entry $Y_{bij}$ represents the score of an arc from a head word at position&nbsp;$j$ to a dependent at position&nbsp;$i$ in the $b$th sentence of the batch. Note that the number of possible heads is one greater than the number of possible dependents because the possible heads include the root of the dependency tree, which we represent using the special token `[CLS]` (at position&nbsp;0).

### 🤞 Test your code

To test you code, instantiate the parsing model and feed it a small batch of sentences.

In [20]:
# Test the model
model = DistilBertForParsing.from_pretrained('distilbert-base-uncased')

sentences = TRAIN_DATA[531:534]

encoded_inputs: BatchEncoding = []
token_spans_list: list[list[TokenSpan]] = []

for sentence in sentences:
    print(sentence)

encoded_inputs, token_spans_list = encode(TOKENIZER, sentences)

out = model.forward(encoded_inputs, token_spans_list)

print(out.shape) # should be (1, 5, 6)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForParsing were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['biaffine.weight', 'biaffine.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-76e280732ca7>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens.input_ids = torch.tensor(tokens.input_ids)


[('I', 2), ('like', 0), ('yuor', 4), ('blog', 2), ('.', 2)]
[('Please', 2), ('check', 0), ('out', 2), ('my', 10), ('who', 6), ('let', 10), ('the', 8), ('dog', 6), ('out', 6), ('blog', 2), ('.', 2)]
[('hey', 6), ('that', 6), ("'s", 6), ('a', 6), ('great', 6), ('blog', 0), (',', 10), ('I', 10), ('also', 10), ('have', 6), ('a', 12), ('site', 10), ('about', 17), ('los', 15), ('angeles', 17), ('online', 17), ('dating', 12)]
torch.Size([3, 17, 18])


## Data loader

We are now almost ready to train the parser. The missing piece is a data collator that prepares a batch of parsed sentences:

* tokenises the sentences and extracts token spans using `encode()` (Problem&nbsp;1)
* constructs the ground-truth head tensor needed to compute the loss (Problem&nbsp;2)

The code in the next cell implements these two steps. For pseudo-words introduced through padding, we assign a head position of −100. This value is ignored by PyTorch’s cross-entropy loss function.

In [21]:
import torch

class ParserBatcher(object):

    def __init__(self, tokenizer, device=None):
        self.tokenizer = tokenizer
        self.device = device

    def __call__(self, parser_inputs):
        encoded_input, start_indices = encode(self.tokenizer, parser_inputs)

        # Get the maximal number of words, for padding
        max_num_words = max(len(s) for s in parser_inputs)

        # Construct tensor containing the ground-truth heads
        all_heads = []
        for parser_input in parser_inputs:
            words, heads = zip(*parser_input)
            heads = list(heads)
            heads.extend([-100] * (max_num_words - len(heads)))  # -100 will be ignored
            all_heads.append(heads)
        all_heads = torch.LongTensor(all_heads)

        # Send all data to the specified device
        if self.device:
            encoded_input = encoded_input.to(self.device)
            all_heads = all_heads.to(self.device)

        return encoded_input, start_indices, all_heads

## Training loop

Finally, here is the training loop of the parser. Most of it is quite standard. The training loss of the parser is the cross-entropy between the head scores and the ground truth head positions. In other words, the parser is trained as a classifier that predicts the position of each word&rsquo;s head.

In [22]:
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# DEVICE = torch.device('mps')

def train(dataset, n_epochs=1, lr=1e-5, batch_size=8):
    # Initialise the tokeniser
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

    # Initialise the encoder
    model = DistilBertForParsing.from_pretrained('distilbert-base-uncased').to(DEVICE)

    # Initialise the data loader
    data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=ParserBatcher(tokenizer, device=DEVICE))

    # Initialise the optimiser
    optimizer = Adam(model.parameters(), lr=lr)

    # Train for the specified number of epochs
    for epoch in range(n_epochs):
        model.train()

        # We keep track of the running loss
        running_loss = 0
        n_batches = 0
        with tqdm(total=len(dataset)) as pbar:
            pbar.set_description(f'Epoch {epoch+1}')

            # Process a batch of samples
            for encoded_input, token_spans, gold_heads in data_loader:
                optimizer.zero_grad()

                # Compute the arc scores
                arc_scores = model.forward(encoded_input, token_spans)
                # shape: [batch_size, num_words, num_words+1]

                # Flatten arc_scores and gold_heads for cross_entropy
                loss = F.cross_entropy(arc_scores.flatten(0, -2), gold_heads.view(-1))
                # shape of the flattened arc_scores: [batch_size * num_words, num_words+1]
                # shape of the flattened gold_heads: [batch_size * num_words]

                # Backward pass
                loss.backward()
                optimizer.step()

                # Update the loss
                running_loss += loss.item()
                n_batches += 1
                pbar.set_postfix(loss=running_loss/n_batches)
                pbar.update(len(token_spans))

    return model

We are now ready to train the parser. With a GPU, you should expect training times of approximately 3&nbsp;minutes per epoch.

In [23]:
PARSING_MODEL = train(TRAIN_DATA, n_epochs=15)

Some weights of DistilBertForParsing were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['biaffine.weight', 'biaffine.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1:   0%|          | 0/12544 [00:00<?, ?it/s]<ipython-input-13-76e280732ca7>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens.input_ids = torch.tensor(tokens.input_ids)
Epoch 15: 100%|██████████| 12544/12544 [01:57<00:00, 106.79it/s, loss=0.286]


## Problem 5: Evaluation (2 points)

Dependency parsers are commonly evaluated using **unlabelled attachment score (UAS)**, which is the percentage of (non-root) words that have been assigned their correct heads. The following cell contains skeleton code for a function `uas()` that computes this score on a given dataset.

In [27]:
def uas(tokenizer, model, filename, batch_size=8):
    # Compute score on a given dataset
    dataset: ParserDataset = ParserDataset(filename)
    data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=ParserBatcher(tokenizer, device=DEVICE))
    model.eval()

    total_correct = 0
    total_words = 0

    for batch in data_loader:
        encoded_inputs, token_spans, gold_heads = batch
        input_ids = encoded_inputs['input_ids']
        attention_mask = encoded_inputs['attention_mask']

        # Forward pass: compute predictions by the model
        outputs = model.forward(encoded_inputs, token_spans)
        predicted_heads = torch.argmax(outputs, dim=-1)

        # Calculate correctness
        correct = (predicted_heads == gold_heads)
        total_correct += correct.sum().item()
        total_words += gold_heads.ne(-100).sum().item()-2 # Count all actual words and exclude padding-words

    # Compute Unlabeled Attachment Score
    uas_score = total_correct / total_words if total_words > 0 else 0
    return uas_score

Implement the `uas()` function to match the following specification:

**uas** (*tokenizer*, *model*, *filename*)

> Takes a tokenizer (*tokenizer*), a trained parsing model (*model*), and the filename of a dataset in the CoNLLU format (*filename*) and returns the unlabelled attachment score of the model on the tokenised dataset.

**⚠️ Note that pseudo-words corresponding to padding must be excluded from the calculation of the UAS.**

The code in the following cell evaluates the trained parser on the development section of the data:

In [28]:
uas(TOKENIZER, PARSING_MODEL, 'en_ewt-ud-dev.conllu')

<ipython-input-13-76e280732ca7>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens.input_ids = torch.tensor(tokens.input_ids)


0.8650951434251634

**Your notebook must contain output demonstrating at least 86% UAS on the development data.**

## Problem 6: Counting trees (6 points)

In the last problem of this lab, we ask you to take a closer look at the graph-theoretic properties of the outputs of your trained model.

The following cell contains code that will use the tokeniser and your trained model to parse the development data and write the gold-standard dependency trees as well as the dependency trees predicted by the parser to a JSON file:

In [29]:
import json

dataset = ParserDataset('en_ewt-ud-dev.conllu')
data_loader = DataLoader(dataset, collate_fn=ParserBatcher(TOKENIZER, device=DEVICE))

with open('graphs.jsonl', 'wt') as fp:
    for encoded_input, token_spans, gold_heads in data_loader:
        with torch.no_grad():
            head_scores = PARSING_MODEL.forward(encoded_input, token_spans)
            pred_heads = torch.argmax(head_scores, dim=-1)
        for gold, pred in zip(gold_heads, pred_heads):
            mask = gold.ne(-100)
            print(json.dumps((gold[mask].tolist(), pred[mask].tolist())), file=fp)

<ipython-input-13-76e280732ca7>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens.input_ids = torch.tensor(tokens.input_ids)


The lab directory contains a script `analyze.py` that you can use to compute statistics on various graph-theoretic properties: total number of graphs, number of acyclic graphs, connected graphs, trees, and projective trees. Document your exploration in a short reflection piece (ca.&nbsp;150 words). Respond to the following prompts

* What are the results of this exploration? Summarise the statistics you obtained.
* Based on what you know about the parser, did you expect your results? Was there anything surprising in them?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

We expect there to be no cycles in any of the graphs since we should only have one root. Furthermore all graphs should be connected since any word in the sentence should be reachable from the root. That also means that we expect all sentences to be trees since you should be able to reach any word in the sentence from the root. 

we got the following results: 

{'total': 2001, 'gold': {'acyclic': 2001, 'connected': 2001, 'tree': 2001, 'projective': 1959}, 'pred': {'acyclic': 2001, 'connected': 1840, 'tree': 1840, 'projective': 1753}}

We can see here that all sentences were indeed acyclic which is inline with expectations. However, not as many sentences were connected and thus also not trees. 

We expected the parser to have some imperfections since it still is stochastic, although our implementation seems to have been on the lower end compared to the bench-marks presented in the slides. This could be due to insufficient amount of training or some inefficiency in the implementation. 

We learned about how to implement a dependency parser by implementing it and subsequently evaluating its performance. This is important since it not only gives an insight into how dependecy parsing works, but also acts as a general introduction to the field of syntactic parsing which will be useful for future studies. An example in the near future being the project. 

**🥳 Congratulations on finishing this lab! 🥳**